In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
import numpy as np
import pandas as pd
from torch.utils.data import Dataset
import math

In [2]:
dataset = pd.read_csv('train_data/train_data.csv')

In [3]:
labels = pd.read_csv('train_data/train_labels.csv')

In [5]:
timeint = pd.to_datetime(dataset['S_2']).astype(int)

In [6]:
timeint = (timeint-timeint.min())/(timeint.max()-timeint.min())

In [7]:
dataset_ = dataset.drop(['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68', 'S_2'], axis=1)

In [8]:
dataset_ = pd.concat([dataset_, timeint], axis = 1)

In [9]:
dataset_ = dataset_.fillna(0)

In [10]:
dataset_ = dataset_.groupby(["customer_ID"], as_index=False).agg(list)

In [14]:
def fillSeries(row):
    new_row = []
    size = len(row[1])
    if size < 13:
        for idx in range(len(row)):
            if idx == 0: new_row.append(row[idx])
            else: new_row.append([row[idx][0] for cnt in range(13 - size)] + row[idx])
    else:
        new_row = row
    return new_row

In [14]:
dataset_ = dataset_.apply(fillSeries, axis=1)

In [15]:
dataset_ = dataset_.set_index('customer_ID').join(labels.set_index('customer_ID'))

In [16]:
dataset_ = dataset_.reset_index(level=0)

In [17]:
dataset_.to_pickle("./transformed_dataset")

In [7]:
dummies = pd.get_dummies(dataset[['D_63', 'D_64']])

In [9]:
dummies = pd.concat([dataset[['customer_ID', 'B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_68']], dummies], axis=1)

In [11]:
dummies = dummies.groupby(["customer_ID"], as_index=False).agg(list)

In [15]:
dummies = dummies.apply(fillSeries, axis=1)

In [16]:
dummies = dummies.set_index('customer_ID').join(labels.set_index('customer_ID'))

In [17]:
dummies.to_pickle("./categorical_dataset")

In [19]:
def metrics(row):
    new_row = []
    for idx in range(len(row)):
        if idx == 0: new_row.append(row[idx])
        else:
            new_row.append(np.average(row[idx]))
            new_row.append(np.mean(row[idx]))
            new_row.append(np.std(row[idx]))
            new_row.append(np.var(row[idx]))
    return new_row

In [18]:
dummies.shape

(458913, 19)

In [20]:
#dataset_ = dataset_.apply(metrics, axis=1)

In [21]:
#dataset_.to_pickle("./transformed_dataset_metrics")